In [1]:
from speechbrain.lobes.models.ECAPA_TDNN import ECAPA_TDNN

In [2]:
import os                       # for working with files

import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
from PIL import Image
import json

import torch                    # Pytorch module 
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import Dataset, DataLoader # for dataloaders 
import torchvision.transforms as transforms   # for transforming images into tensors 
from torchvision import datasets,transforms,models   # for working with classes and images
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torchsummary import summary              # for getting the summary of our model

import speechbrain as sb
from torch.nn.utils.rnn import pad_sequence


c:\Python312\Lib\inspect.py:1004: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


KeyboardInterrupt: 

In [ ]:
class ECAPAEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = ECAPA_TDNN(input_size=80, lin_neurons=192)

    def forward(self, features):  # features: [B, T, F]
        return self.model(features)  # output: x-vector [B, 192]

In [ ]:
from speechbrain.lobes.features import Fbank

fbank_extractor = Fbank(n_mels=80)

@sb.utils.data_pipeline.takes("sig")
@sb.utils.data_pipeline.provides("fbank")
def compute_fbank(sig):
    fbank = fbank_extractor(sig)
    return fbank

In [ ]:
waveform = torch.randn(3 *16000)  # 1 giây @ 16kHz
waveform = torch.stack([waveform, waveform], dim = 0)
print("waveform shape:", waveform.shape)
fbank_extractor = Fbank(n_mels=80)
fbank_feat = fbank_extractor(waveform)  # [1, T, 80]

print("Fbank shape:", fbank_feat.shape)

encoder = ECAPAEncoder()
with torch.no_grad():
    output = encoder(fbank_feat)  # [1, 192]

print("Fbank shape:", fbank_feat.shape)
print("ECAPA x-vector shape:", output.shape)

waveform shape: torch.Size([2, 48000])
Fbank shape: torch.Size([2, 301, 80])
Fbank shape: torch.Size([2, 301, 80])
ECAPA x-vector shape: torch.Size([2, 1, 192])


In [ ]:
import os
print(os.path.exists("D:/home4/vuhl/VSASV-Dataset/vlsp2025/train"))

True


In [ ]:
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
signal, fs =torchaudio.load("data/00000.wav")
embeddings = classifier.encode_batch(signal)

In [ ]:
embeddings.squeeze().shape

torch.Size([192])

In [ ]:
from huggingface_hub import hf_hub_download
import torch

# Tải model file từ repo
ckpt_path = hf_hub_download(repo_id="jungjee/RawNet3", filename="model.pt")

# Load state_dict
state_dict = torch.load(ckpt_path, map_location="cpu")

# In ra các keys để biết tên lớp model nếu cần custom
print(state_dict.keys())

(…)40e56a8756835b1911b04f9c44f347d47c09f90c:   0%|          | 0.00/65.3M [00:00<?, ?B/s]

dict_keys(['model'])


c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--jungjee--RawNet3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
from dataloader import SASVDataset
sasv_dataset = SASVDataset(data_root="D:/home4/vuhl/VSASV-Dataset/vlsp2025/train")

In [ ]:
def custom_collate_fn(batch):
    tst_waveforms = [item['tst_waveform'] for item in batch]
    enr_features = [item['enr_feature'] for item in batch]
    tst_features = [item['tst_feature'] for item in batch]
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.long)

    tst_waveforms = pad_sequence(tst_waveforms, batch_first=True)     # [B, T]
    enr_features = pad_sequence(enr_features, batch_first=True)       # [B, T, F]
    tst_features = pad_sequence(tst_features, batch_first=True)       # [B, T, F]

    return tst_waveforms, enr_features, tst_features, labels

In [ ]:
loader = DataLoader(sasv_dataset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)

In [ ]:
sample = next(iter(loader))

NameError: name 'loader' is not defined

In [ ]:
score, prediction = verification.verify_files("tests/samples/ASR/spk1_snt1.wav", "tests/samples/ASR/spk1_snt2.wav") # Same Speaker
